In [68]:
import os
import pandas as pd

In [69]:
from io import StringIO
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

In [ ]:
service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
container = service_client.get_container_client(os.getenv("CONTAINER_NAME"))

In [79]:
def clean_add_features(df: pd.DataFrame):

    df["Volume"] = df["Volume"].str.replace(',','').astype(int)
    df["Date"] = pd.to_datetime(df["Date"], format='%m/%d/%Y')

    # Dates
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Year"] = df["Date"].dt.year
    df["DayOfWeek"] = df["Date"].dt.dayofweek

    df["MA_5"] = df["Close"].rolling(5).mean()
    df["MA_20"] = df["Close"].rolling(20).mean()
    df["EMA_5"] = df["Close"].ewm(span=5, adjust=False).mean()
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["STD_5"] = df["Close"].rolling(5).mean()
    df["STD_20"] = df["Close"].rolling(20).mean()

    df["DailyReturn"] = df["Close"].pct_change()
    df["Volatility"] = df["DailyReturn"].rolling(20).std()
 
    return df

In [80]:
staging = []
for blob in container.list_blobs(name_starts_with="bronze/bdo.csv"):
    if blob.name.endswith('.csv'):
        blob_client = container.get_blob_client(blob.name)
        content = blob_client.download_blob().readall().decode('utf-8')

        file = pd.read_csv(StringIO(content))
        file['Source'] = os.path.basename(blob.name).replace('.csv', '')
        staging.append(clean_add_features(file))

In [81]:
file.head(1)

,Date,Open,High,Low,Close,Volume,Source,Month,Day,Year,DayOfWeek,MA_5,MA_20,EMA_5,EMA_20,STD_5,STD_20,DailyReturn,Volatility
0,2025-07-24,152.2,153.9,152.0,152.2,1307730,bdo,7,24,2025,3,NaN,NaN,152.2,152.2,NaN,NaN,NaN,NaN


In [82]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         326 non-null    datetime64[ns]
 1   Open         326 non-null    float64       
 2   High         326 non-null    float64       
 3   Low          326 non-null    float64       
 4   Close        326 non-null    float64       
 5   Volume       326 non-null    int64         
 6   Source       326 non-null    object        
 7   Month        326 non-null    int32         
 8   Day          326 non-null    int32         
 9   Year         326 non-null    int32         
 10  DayOfWeek    326 non-null    int32         
 11  MA_5         322 non-null    float64       
 12  MA_20        307 non-null    float64       
 13  EMA_5        326 non-null    float64       
 14  EMA_20       326 non-null    float64       
 15  STD_5        322 non-null    float64       
 16  STD_20  

In [84]:
df = pd.concat(staging, ignore_index=True)

In [85]:
df

,Date,Open,High,Low,Close,Volume,Source,Month,Day,Year,DayOfWeek,MA_5,MA_20,EMA_5,EMA_20,STD_5,STD_20,DailyReturn,Volatility
0,2025-07-24,152.2,153.9,152.0,152.2,1307730,bdo,7,24,2025,3,NaN,NaN,152.200000,152.200000,NaN,NaN,NaN,NaN
1,2025-07-22,149.9,149.9,148.2,149.9,2438500,bdo,7,22,2025,1,NaN,NaN,151.433333,151.980952,NaN,NaN,-0.015112,NaN
2,2025-07-21,150.9,151.0,148.9,151.0,1228600,bdo,7,21,2025,0,NaN,NaN,151.288889,151.887528,NaN,NaN,0.007338,NaN
3,2025-07-17,147.9,149.8,147.5,148.6,2047490,bdo,7,17,2025,3,NaN,NaN,150.392593,151.574430,NaN,NaN,-0.015894,NaN
4,2025-07-16,150.0,151.1,147.2,151.4,3485930,bdo,7,16,2025,2,150.62,NaN,150.728395,151.557818,150.62,NaN,0.018843,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,2024-03-22,154.5,155.1,153.5,154.2,3800680,bdo,3,22,2024,4,154.40,150.530,153.685807,151.001864,154.40,150.530,0.005871,0.016249
322,2024-03-21,154.0,155.6,154.0,154.5,3289060,bdo,3,21,2024,3,153.50,151.050,153.957204,151.335020,153.50,151.050,0.001946,0.015950
323,2024-03-20,153.4,154.8,151.8,154.0,3733010,bdo,3,20,2024,2,153.40,151.570,153.971470,151.588828,153.40,151.570,-0.003236,0.015944
324,2024-03-19,152.9,154.0,151.3,153.5,2238910,bdo,3,19,2024,1,153.90,151.905,153.814313,151.770844,153.90,151.905,-0.003247,0.015384
